<h1><center>Applied Data Science Capstone Project</center></h1>

In [44]:
import numpy as np # library to handle data in a vectorized manner
import math

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install folium -c conda-forge
import folium

<h1>Get Data From Foursquare API</h1>
<h3>Note:  This code was executed progressively to ensure API limit was not reached</h3>

In [45]:
df = pd.read_csv('geoCities.csv')
df.head()

,NAME,STNAME,POPESTIMATE2017,NAME2,latMin,latMax,longMin,longMax
0,Advance town,Indiana,512,Advance,39.988492,40.001171,-86.629306,-86.612624
1,Akron town,Indiana,1123,Akron,41.033091,41.045082,-86.033180,-86.015913
2,Alamo town,Indiana,65,Alamo,39.981722,39.985384,-87.057804,-87.052821
3,Albany town,Indiana,2169,Albany,40.295984,40.321093,-85.254026,-85.214599
4,Albion town,Indiana,2327,Albion,41.377984,41.411133,-85.438043,-85.395647


In [46]:
def getGrid(latMin,latMax,longMin,longMax,radius):
    radius=radius/1609  #meters convert to miles
    gridList=[]
    segLength=(2**0.5)*radius
    latIncrement=segLength/69
    numberLatPoints=math.ceil((latMax-latMin)*69/segLength)
    averageLatRadians=(latMin+latMax)/2/180
    numberLongPoints=math.ceil((longMax-longMin)*(69*math.cos(averageLatRadians))/segLength)
    longIncrement=segLength/(math.cos(averageLatRadians)*69)
    
    for i in range(numberLatPoints):
        for j in range(numberLongPoints):
            lat=latMin+i*latIncrement
            long=longMin+j*longIncrement
            gridList.append([lat,long])
    return gridList
            
    

In [47]:
testdf=df[df['POPESTIMATE2017']>20000]
testdf.shape
testdf.reset_index(drop=True,inplace=True)
testdf.head()
testdf.shape

(43, 8)

In [48]:
a={}
radius=500
for i,row in testdf.iterrows():
    a[i]=getGrid(row['latMin'],row['latMax'],row['longMin'],row['longMax'],radius)


In [49]:
count=0
for i in testdf.index:
    count=count+len(a[i])
count

18344

In [76]:
a[15]

[[39.7535519, -85.8306],
 [39.7535519, -85.82407201582714],
 [39.7535519, -85.81754403165428],
 [39.7535519, -85.81101604748142],
 [39.7535519, -85.80448806330855],
 [39.7535519, -85.79796007913569],
 [39.7535519, -85.79143209496283],
 [39.7535519, -85.78490411078997],
 [39.7535519, -85.7783761266171],
 [39.7535519, -85.77184814244424],
 [39.7535519, -85.76532015827138],
 [39.7535519, -85.75879217409852],
 [39.7535519, -85.75226418992565],
 [39.7535519, -85.74573620575279],
 [39.7535519, -85.73920822157993],
 [39.7535519, -85.73268023740707],
 [39.75992102639218, -85.8306],
 [39.75992102639218, -85.82407201582714],
 [39.75992102639218, -85.81754403165428],
 [39.75992102639218, -85.81101604748142],
 [39.75992102639218, -85.80448806330855],
 [39.75992102639218, -85.79796007913569],
 [39.75992102639218, -85.79143209496283],
 [39.75992102639218, -85.78490411078997],
 [39.75992102639218, -85.7783761266171],
 [39.75992102639218, -85.77184814244424],
 [39.75992102639218, -85.76532015827138],


In [51]:
gmap = folium.Map(location=a[0][0], zoom_start=12)

# add markers to map
for point in a[2]:
    
    label = '{}, {}'.format(point[0], point[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    point,
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(gmap)  
    
gmap

In [52]:
# @hidden_cell

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=1000

In [91]:
fbdf=testdf.loc[31:50,:]

In [92]:
for i in fbdf.index:
    print(i)

31
32
33
34
35
36
37
38
39
40
41
42


In [93]:
fbdf.loc[i,"NAME2"]

'Westfield'

In [94]:
venues_list=[]
error_list=[]

testGeo=[[39.608333,-86.31835],[39.928333,-85.99835]]
for i in fbdf.index:
    for point in a[i]:  
        city=fbdf.loc[i,"NAME2"]
        state=fbdf.loc[i,"STNAME"]
        lat=point[0]
        lng=point[1]
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            for v in results:            
                venues_list.append([
                    city,
                    state,
                    lat, 
                    lng,
                    v['venue']['id'],
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']])
        except:
            error_list.append([city,lat,lng,results,url])
        
        
        

        
       

In [95]:
dft=pd.DataFrame(venues_list)
dft.columns=["City",'State','City Lat','City Long','Venue Id','Venue Name','Venue Lat','Venue Lng','Category']
dft.shape



(11577, 9)

In [96]:
len(error_list)

2

In [97]:
dft.drop_duplicates(subset=['Venue Id'],inplace=True)

In [98]:
dft.shape

(6633, 9)

In [102]:
dft[dft['City']=='Westfield'].shape

(448, 9)

In [105]:
d=pd.read_csv('cities31to42.csv')

In [106]:
d.shape

(6633, 9)